In [27]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [ ]:
model_metrics = {}

In [28]:
df = pd.read_csv('rdkit_descriptors_208_with_activity.csv')
df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Activity
0,14.607685,-1.275102,14.607685,0.137208,0.893230,331.347,313.203,331.133220,126,0,...,0,0,0,0,0,0,0,0,0,0
1,10.427986,-0.635752,10.427986,0.196415,0.384874,325.737,317.673,325.003638,108,0,...,0,0,0,0,0,0,0,0,0,0
2,10.505713,-0.632032,10.505713,0.204419,0.353751,350.356,336.244,350.068491,126,0,...,0,0,0,0,0,0,0,0,0,0
3,10.420390,-0.618689,10.420390,0.256214,0.388807,290.304,280.224,290.047361,102,0,...,0,0,0,0,0,0,0,0,0,0
4,10.461498,-0.631261,10.461498,0.201693,0.319099,449.084,442.028,446.863634,114,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
'Smiles' in df.columns


False

In [30]:
# Get columns with null values
null_columns = df.columns[df.isnull().any()].tolist()

# Print the columns with null values
print("Columns with null values:", null_columns)

for i in null_columns:
    print(df[i].isnull().mean()*100)

Columns with null values: ['MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW']
0.3110419906687403
0.3110419906687403
0.3110419906687403
0.3110419906687403
0.44974990542642174
0.44974990542642174
0.44974990542642174
0.44974990542642174
0.44974990542642174
0.44974990542642174
0.44974990542642174
0.44974990542642174


In [31]:
df.shape

(23791, 209)

In [32]:
df.dropna(inplace = True)
df.shape

(23684, 209)

In [33]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler



# Display the shape and summary of the DataFrame before dropping missing values
print(f'Shape before dropping NaN values: {df.shape}')
print(df.describe())

# Drop rows with missing values
df.dropna(inplace=True)
print(f'Shape after dropping NaN values: {df.shape}')

# Prepare features and target variable
X = df.drop(['Activity'], axis=1)
y = df['Activity']
print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')

# Scale the features to be between 0 and 1
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Print the statistics of X after scaling
print("Statistics of X after Min-Max scaling (0 to 1):")
print(X_scaled.describe())

# Include target variable and update df
df = X_scaled.copy()
df['Activity'] = y.reset_index(drop=True)

# Display the updated DataFrame
print("Updated DataFrame with scaled features and activity:")
print(df.head())


Shape before dropping NaN values: (23684, 209)
       MaxEStateIndex  MinEStateIndex  MaxAbsEStateIndex  MinAbsEStateIndex  \
count    23684.000000    23684.000000       23684.000000       23684.000000   
mean        11.598384       -1.032131          11.598384           0.171020   
std          2.700752        1.537435           2.700752           0.189650   
min          2.162037       -8.728814           2.162037           0.000000   
25%         11.266432       -1.207852          11.266432           0.043037   
50%         12.394367       -0.510455          12.394367           0.111394   
75%         13.086184       -0.182188          13.086184           0.221099   
max         16.689937        1.875000          16.689937           1.875000   

                qed         MolWt  HeavyAtomMolWt    ExactMolWt  \
count  23684.000000  23684.000000    23684.000000  23684.000000   
mean       0.495343    411.768541      389.802619    411.255393   
std        0.205891    162.321989      1

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterSampler

# Define the neural network model
class ANNModel(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_layer_sizes[0])
        self.fc2 = nn.Linear(hidden_layer_sizes[0], hidden_layer_sizes[1])
        self.fc3 = nn.Linear(hidden_layer_sizes[1], hidden_layer_sizes[2])
        self.fc4 = nn.Linear(hidden_layer_sizes[2], 1)  # Output layer for binary classification
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

# Function to train the model
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(train_loader)

# Function to evaluate the model
def evaluate_model(model, test_loader, device):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            predicted = (outputs >= 0.5).float()  # Apply threshold for binary classification
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    return y_true, y_pred

# Assume df_maccs is your DataFrame
# Replace with the actual DataFrame loading code
# df_maccs = ...

X = df.drop('Activity', axis=1)
y = df['Activity']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25, random_state=69)

# Convert pandas DataFrame/Series to NumPy arrays
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()
X_test_array = X_test.to_numpy()
y_test_array = y_test.to_numpy()

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_array, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_array, dtype=torch.float32).unsqueeze(1)

# Define the hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(128, 256, 128)],
    'optimizer': ['sgd', 'adam', 'lbfgs'],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30]
}

# Generate parameter combinations using ParameterSampler
param_list = list(ParameterSampler(param_grid, n_iter=50, random_state=42))

# Best accuracy and parameters tracker
best_acc = 0
best_params = None

# Loop through each parameter set
for params in param_list:
    print(f"Testing parameters: {params}")

    model = ANNModel(input_size=X_train_tensor.shape[1], hidden_layer_sizes=params['hidden_layer_sizes']).to('cpu')

    # Optimizer selection
    if params['optimizer'] == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=params['learning_rate'])
    elif params['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    elif params['optimizer'] == 'lbfgs':
        optimizer = optim.LBFGS(model.parameters(), lr=params['learning_rate'])

    # Define the loss function
    criterion = nn.BCELoss()

    # Create DataLoader for training and testing
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=params['batch_size'], shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=params['batch_size'], shuffle=False)

    # Training loop
    for epoch in range(params['epochs']):
        if params['optimizer'] == 'lbfgs':
            # Define closure for LBFGS
            def closure():
                optimizer.zero_grad()
                outputs = model(X_train_tensor)
                loss = criterion(outputs, y_train_tensor)
                loss.backward()
                return loss

            optimizer.step(closure)

        else:
            running_loss = train_model(model, criterion, optimizer, train_loader, 'cpu')
            print(f'Epoch [{epoch+1}/{params["epochs"]}], Loss: {running_loss:.4f}')

    # Evaluate the model
    y_true, y_pred = evaluate_model(model, test_loader, 'cpu')
    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_params = params

print(f"Best Accuracy: {best_acc}")
print(f"Best Parameters: {best_params}")


Testing parameters: {'optimizer': 'sgd', 'learning_rate': 0.01, 'hidden_layer_sizes': (128, 256, 128), 'epochs': 10, 'batch_size': 64}
Epoch [1/10], Loss: 0.6406
Epoch [2/10], Loss: 0.5478
Epoch [3/10], Loss: 0.4941
Epoch [4/10], Loss: 0.4651
Epoch [5/10], Loss: 0.4516
Epoch [6/10], Loss: 0.4470
Epoch [7/10], Loss: 0.4453
Epoch [8/10], Loss: 0.4448
Epoch [9/10], Loss: 0.4445
Epoch [10/10], Loss: 0.4451
Accuracy: 0.8408
Testing parameters: {'optimizer': 'sgd', 'learning_rate': 0.001, 'hidden_layer_sizes': (128, 256, 128), 'epochs': 10, 'batch_size': 32}
Epoch [1/10], Loss: 0.7009
Epoch [2/10], Loss: 0.6682
Epoch [3/10], Loss: 0.6391
Epoch [4/10], Loss: 0.6129
Epoch [5/10], Loss: 0.5894
Epoch [6/10], Loss: 0.5681
Epoch [7/10], Loss: 0.5489
Epoch [8/10], Loss: 0.5316
Epoch [9/10], Loss: 0.5161
Epoch [10/10], Loss: 0.5069
Accuracy: 0.8408
Testing parameters: {'optimizer': 'adam', 'learning_rate': 0.01, 'hidden_layer_sizes': (128, 256, 128), 'epochs': 30, 'batch_size': 32}
Epoch [1/30], Los

In [35]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import warnings
warnings.filterwarnings('ignore')


# Separate features and target variable
X = df.drop('Activity', axis=1)
y = df['Activity']

X = X.clip(lower=-1e6, upper=1e6)

# Split data into training and testing sets (one-time split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Define the number of folds for cross-validation
n_splits = 10

# Define the parameter grid for hyperparameter tuning (smaller range for faster results)
param_dist = {
    'n_estimators': np.arange(50, 201, 10),  # Reduced range for faster computation
    'max_depth': np.arange(5, 21, 1),        # Reduced range for faster computation
    'random_state': [42]                    # Fixed for reproducibility
}

# Create the Random Forest model
rf = RandomForestClassifier()

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=20, cv=n_splits, scoring='roc_auc', n_jobs=-1, random_state=42, verbose=2)

# Perform RandomizedSearchCV to find the best parameters
random_search.fit(X_train, y_train)

# Get the best parameters and train the final Random Forest model
best_params = random_search.best_params_

# Train the Random Forest model using the best parameters
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)

# Make predictions on the test set using the best model
y_pred = best_rf.predict(X_test)
y_pred_prob = best_rf.predict_proba(X_test)[:, 1]  # Probabilities for ROC AUC

# Compute metrics with the final model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
conf_matrix = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

# Extract values from confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Additional metrics
specificity = TN / (TN + FP)
balanced_accuracy = (recall + specificity) / 2
fpr = FP / (FP + TN)  # False Positive Rate

# Store the metrics for the final model in the global dictionary
model_metrics['RandomForest'] = [
    accuracy, roc_auc, precision, recall, f1, specificity, balanced_accuracy, fpr, mcc, [TN, FP, FN, TP]
]

# Beautified printing of metrics
print("\n")
print("="*40)
print("Model Evaluation Metrics".center(40))
print("="*40)
print(f"{'Metric':<20} {'Value':>18}")
print("-"*40)
print(f"{'Accuracy':<20} {accuracy:>18.4f}")
print(f"{'ROC AUC':<20} {roc_auc:>18.4f}")
print(f"{'Precision':<20} {precision:>18.4f}")
print(f"{'Recall':<20} {recall:>18.4f}")
print(f"{'F1 Score':<20} {f1:>18.4f}")
print(f"{'Specificity':<20} {specificity:>18.4f}")
print(f"{'Balanced Accuracy':<20} {balanced_accuracy:>18.4f}")
print(f"{'FPR':<20} {fpr:>18.4f}")
print(f"{'Matthews CC':<20} {mcc:>18.4f}")
print("\n")
print("="*40)
print("Confusion Matrix".center(40))
print("="*40)
print(f"{'TN':<10} {'FP':<10} {'FN':<10} {'TP':<10}")
print("-"*40)
print(f"{TN:<10} {FP:<10} {FN:<10} {TP:<10}")
print("="*40)
print("\n")


Fitting 10 folds for each of 20 candidates, totalling 200 fits
Best parameters: {'random_state': 42, 'n_estimators': 190, 'max_depth': 18}


        Model Evaluation Metrics        
Metric                            Value
----------------------------------------
Accuracy                         0.8769
ROC AUC                          0.8861
Precision                        0.7570
Recall                           0.3776
F1 Score                         0.5038
Specificity                      0.9760
Balanced Accuracy                0.6768
FPR                              0.0240
Matthews CC                      0.4774


            Confusion Matrix            
TN         FP         FN         TP        
----------------------------------------
3858       95         488        296       




In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Separate features and target variable
X = df.drop('Activity', axis=1)
y = df['Activity']

X = X.clip(lower=-1e6, upper=1e6)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Define the parameter distribution for hyperparameter tuning
param_dist = {
    'n_estimators': randint(50, 301),  # Random integer range for n_estimators
    'max_depth': randint(1, 31),       # Random integer range for max_depth
    'scale_pos_weight': uniform(0.1, 10.0)  # Uniform distribution for scale_pos_weight
}

# Create the XGBClassifier model
xgb = XGBClassifier(objective='binary:logistic', random_state=42, use_label_encoder=False, eval_metric='logloss')

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_dist, n_iter=20, random_state=42, cv=5, n_jobs=-1, verbose=2, scoring='roc_auc')

# Perform hyperparameter tuning
random_search.fit(X_train, y_train)

# Get the best parameters from RandomizedSearchCV
best_params = random_search.best_params_

# Train the XGBClassifier model on the entire training set using the best parameters
best_xgb = XGBClassifier(**best_params, objective='binary:logistic', random_state=42, use_label_encoder=False, eval_metric='logloss')
best_xgb.fit(X_train, y_train)

# Make predictions with the best model
y_pred = best_xgb.predict(X_test)
y_pred_prob = best_xgb.predict_proba(X_test)[:, 1]  # Probabilities for ROC AUC

# Compute metrics with the final model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
conf_matrix = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

# Extract values from confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Additional metrics
specificity = TN / (TN + FP)
balanced_accuracy = (recall + specificity) / 2
fpr = FP / (FP + TN)  # False Positive Rate

# Add final model metrics to the global dictionary
model_metrics['XGBoost_final'] = [
    accuracy, roc_auc, precision, recall, f1, specificity, balanced_accuracy, fpr, mcc, [TN, FP, FN, TP]
]

# Beautified printing of metrics
print("\n")
print("="*40)
print("Model Evaluation Metrics".center(40))
print("="*40)
print(f"{'Metric':<20} {'Value':>18}")
print("-"*40)
print(f"{'Accuracy':<20} {accuracy:>18.4f}")
print(f"{'ROC AUC':<20} {roc_auc:>18.4f}")
print(f"{'Precision':<20} {precision:>18.4f}")
print(f"{'Recall':<20} {recall:>18.4f}")
print(f"{'F1 Score':<20} {f1:>18.4f}")
print(f"{'Specificity':<20} {specificity:>18.4f}")
print(f"{'Balanced Accuracy':<20} {balanced_accuracy:>18.4f}")
print(f"{'FPR':<20} {fpr:>18.4f}")
print(f"{'Matthews CC':<20} {mcc:>18.4f}")
print("\n")
print("="*40)
print("Confusion Matrix".center(40))
print("="*40)
print(f"{'TN':<10} {'FP':<10} {'FN':<10} {'TP':<10}")
print("-"*40)
print(f"{TN:<10} {FP:<10} {FN:<10} {TP:<10}")
print("="*40)
print("\n")


In [36]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
import warnings
warnings.filterwarnings('ignore')

# Separate features and target variable
X = df.drop('Activity', axis=1)
y = df['Activity']

X = X.clip(lower=-1e6, upper=1e6)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Define the parameter distribution for hyperparameter tuning
param_dist = {
    'max_depth': randint(1, 31),  # Random integer range for max_depth
    'min_samples_split': randint(2, 20),  # Random integer range for min_samples_split
    'min_samples_leaf': randint(1, 20)  # Random integer range for min_samples_leaf
}

# Create the DecisionTreeClassifier model
dt = DecisionTreeClassifier(random_state=69)

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=dt, param_distributions=param_dist, n_iter=20, cv=10, scoring='roc_auc', random_state=42, n_jobs=-1)

# Fit the RandomizedSearchCV to find the best parameters
random_search.fit(X_train, y_train)

# Get the best parameters from RandomizedSearchCV
best_params = random_search.best_params_

# Train the DecisionTreeClassifier model on the entire training set using the best parameters
best_dt = DecisionTreeClassifier(**best_params, random_state=69)
best_dt.fit(X_train, y_train)

# Make predictions with the best model
y_pred = cross_val_predict(best_dt, X_train, y_train, cv=10)
y_pred_prob = best_dt.predict_proba(X_test)[:, 1]  # Probabilities for ROC AUC

# Compute metrics with the final model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
conf_matrix = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

# Extract values from confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Additional metrics
specificity = TN / (TN + FP)
balanced_accuracy = (recall + specificity) / 2
fpr = FP / (FP + TN)  # False Positive Rate

# Add final model metrics to the global dictionary
model_metrics['DecisionTree'] = [
    accuracy, roc_auc, precision, recall, f1, specificity, balanced_accuracy, fpr, mcc, [TN, FP, FN, TP]
]

# Beautified printing of metrics
print("\n")
print("="*40)
print("Model Evaluation Metrics".center(40))
print("="*40)
print(f"{'Metric':<20} {'Value':>18}")
print("-"*40)
print(f"{'Accuracy':<20} {accuracy:>18.4f}")
print(f"{'ROC AUC':<20} {roc_auc:>18.4f}")
print(f"{'Precision':<20} {precision:>18.4f}")
print(f"{'Recall':<20} {recall:>18.4f}")
print(f"{'F1 Score':<20} {f1:>18.4f}")
print(f"{'Specificity':<20} {specificity:>18.4f}")
print(f"{'Balanced Accuracy':<20} {balanced_accuracy:>18.4f}")
print(f"{'FPR':<20} {fpr:>18.4f}")
print(f"{'Matthews CC':<20} {mcc:>18.4f}")
print("\n")
print("="*40)
print("Confusion Matrix".center(40))
print("="*40)
print(f"{'TN':<10} {'FP':<10} {'FN':<10} {'TP':<10}")
print("-"*40)
print(f"{TN:<10} {FP:<10} {FN:<10} {TP:<10}")
print("="*40)
print("\n")


Fitting 10 folds for each of 20 candidates, totalling 200 fits
Best parameters for Decision Tree: {'max_depth': 20, 'min_samples_leaf': 15, 'min_samples_split': 8}


        Model Evaluation Metrics        
Metric                            Value
----------------------------------------
Accuracy                         0.8493
ROC AUC                          0.7752
Precision                        0.5658
Recall                           0.3839
F1 Score                         0.4574
Specificity                      0.9416
Balanced Accuracy                0.6627
FPR                              0.0584
Matthews CC                      0.3831


            Confusion Matrix            
TN         FP         FN         TP        
----------------------------------------
3722       231        483        301       


[CV] END max_depth=7, min_samples_leaf=15, min_samples_split=12; total time=   2.7s
[CV] END max_depth=7, min_samples_leaf=15, min_samples_split=12; total time=   2.6s
[CV] END m

In [37]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
import warnings
warnings.filterwarnings('ignore')

# Separate features and target variable
X = df.drop('Activity', axis=1)
y = df['Activity']

X = X.clip(lower=-1e6, upper=1e6)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Define the parameter distribution for hyperparameter tuning
param_dist = {
    'C': uniform(0.01, 10.0),  # Uniform distribution for regularization strength
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],  # Regularization penalty
    'solver': ['lbfgs', 'liblinear', 'saga']  # Algorithms for optimization
}

# Create the LogisticRegression model
logreg = LogisticRegression(random_state=69, max_iter=200)

# Use RandomizedSearchCV to search for the best hyperparameters
random_search = RandomizedSearchCV(
    estimator=logreg,
    param_distributions=param_dist,
    n_iter=20,  # Number of iterations
    random_state=42,
    scoring='roc_auc',  # Use ROC AUC as scoring metric
    cv=5,  # 5-fold cross-validation
    n_jobs=-1
)

# Fit the model using RandomizedSearchCV
random_search.fit(X_train, y_train)

# Extract the best parameters and metrics
best_params = random_search.best_params_

# Train the final LogisticRegression model with the best hyperparameters
best_logreg = LogisticRegression(**best_params, random_state=69, max_iter=200)
best_logreg.fit(X_train, y_train)

# Make predictions with the best model
y_pred = best_logreg.predict(X_test)
y_pred_prob = best_logreg.predict_proba(X_test)[:, 1]  # Probabilities for ROC AUC

# Compute the metrics for the final model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
conf_matrix = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

# Extract values from the confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Additional metrics
specificity = TN / (TN + FP)
balanced_accuracy = (recall + specificity) / 2
fpr = FP / (FP + TN)  # False Positive Rate

# Add final model metrics to the global dictionary as a list
model_metrics['LogisticRegression'] = [
    accuracy, roc_auc, precision, recall, f1, specificity, balanced_accuracy, fpr, mcc, [TN, FP, FN, TP]
]

# Beautified printing of metrics
print("\n")
print("="*40)
print("Model Evaluation Metrics".center(40))
print("="*40)
print(f"{'Metric':<20} {'Value':>18}")
print("-"*40)
print(f"{'Accuracy':<20} {accuracy:>18.4f}")
print(f"{'ROC AUC':<20} {roc_auc:>18.4f}")
print(f"{'Precision':<20} {precision:>18.4f}")
print(f"{'Recall':<20} {recall:>18.4f}")
print(f"{'F1 Score':<20} {f1:>18.4f}")
print(f"{'Specificity':<20} {specificity:>18.4f}")
print(f"{'Balanced Accuracy':<20} {balanced_accuracy:>18.4f}")
print(f"{'FPR':<20} {fpr:>18.4f}")
print(f"{'Matthews CC':<20} {mcc:>18.4f}")
print("\n")
print("="*40)
print("Confusion Matrix".center(40))
print("="*40)
print(f"{'TN':<10} {'FP':<10} {'FN':<10} {'TP':<10}")
print("-"*40)
print(f"{TN:<10} {FP:<10} {FN:<10} {TP:<10}")
print("="*40)
print("\n")


Fitting 10 folds for each of 20 candidates, totalling 200 fits


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warni

Best parameters: {'C': 8.09397348116461, 'penalty': 'l1', 'solver': 'liblinear'}


        Model Evaluation Metrics        
Metric                            Value
----------------------------------------
Accuracy                         0.8457
ROC AUC                          0.7541
Precision                        0.6293
Recall                           0.1645
F1 Score                         0.2609
Specificity                      0.9808
Balanced Accuracy                0.5727
FPR                              0.0192
Matthews CC                      0.2654


            Confusion Matrix            
TN         FP         FN         TP        
----------------------------------------
3877       76         655        129       




In [38]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from sklearn.naive_bayes import GaussianNB
from scipy.stats import uniform
import warnings
warnings.filterwarnings('ignore')

# Separate features and target variable
X = df.drop('Activity', axis=1)
y = df['Activity']

X = X.clip(lower=-1e6, upper=1e6)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Define the parameter distribution for hyperparameter tuning
param_dist = {
    'var_smoothing': uniform(1e-10, 1e-9)  # Uniform distribution for var_smoothing
}

# Create the GaussianNB model
gnb = GaussianNB()

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=gnb, param_distributions=param_dist, n_iter=200, scoring='roc_auc', cv=5, random_state=42, n_jobs=-1)

# Fit the model with the best parameters
random_search.fit(X_train, y_train)

# Get the best model from RandomizedSearchCV
best_gnb = random_search.best_estimator_

# Make predictions with the best model
y_pred = best_gnb.predict(X_test)
y_pred_prob = best_gnb.predict_proba(X_test)[:, 1]  # Probabilities for ROC AUC

# Compute metrics with the final model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
conf_matrix = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

# Extract values from confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Additional metrics
specificity = TN / (TN + FP)
balanced_accuracy = (recall + specificity) / 2
fpr = FP / (FP + TN)  # False Positive Rate

# Add final model metrics to the global dictionary
model_metrics['GaussianNB'] = [
    accuracy,
    roc_auc,
    precision,
    recall,
    f1,
    specificity,
    balanced_accuracy,
    fpr,
    mcc,
    [TN, FP, FN, TP]
]

# Beautified printing of metrics
print("\n")
print("="*40)
print("Model Evaluation Metrics".center(40))
print("="*40)
print(f"{'Metric':<20} {'Value':>18}")
print("-"*40)
print(f"{'Accuracy':<20} {accuracy:>18.4f}")
print(f"{'ROC AUC':<20} {roc_auc:>18.4f}")
print(f"{'Precision':<20} {precision:>18.4f}")
print(f"{'Recall':<20} {recall:>18.4f}")
print(f"{'F1 Score':<20} {f1:>18.4f}")
print(f"{'Specificity':<20} {specificity:>18.4f}")
print(f"{'Balanced Accuracy':<20} {balanced_accuracy:>18.4f}")
print(f"{'FPR':<20} {fpr:>18.4f}")
print(f"{'Matthews CC':<20} {mcc:>18.4f}")
print("\n")
print("="*40)
print("Confusion Matrix".center(40))
print("="*40)
print(f"{'TN':<10} {'FP':<10} {'FN':<10} {'TP':<10}")
print("-"*40)
print(f"{TN:<10} {FP:<10} {FN:<10} {TP:<10}")
print("="*40)
print("\n")


Fitting 10 folds for each of 200 candidates, totalling 2000 fits
[CV] END ......C=3.7554011884736247, penalty=l1, solver=saga; total time=  28.6s
[CV] END ......C=3.7554011884736247, penalty=l1, solver=saga; total time=  28.7s
[CV] END ......C=3.7554011884736247, penalty=l1, solver=saga; total time=  29.0s
[CV] END .....C=6.021150117432088, penalty=none, solver=saga; total time=  22.5s
[CV] END .....C=6.021150117432088, penalty=none, solver=saga; total time=  22.1s
[CV] END C=0.21584494295802448, penalty=l2, solver=liblinear; total time=   1.5s
[CV] END C=0.21584494295802448, penalty=l2, solver=liblinear; total time=   1.6s
[CV] END C=0.21584494295802448, penalty=l2, solver=liblinear; total time=   1.5s
[CV] END C=0.21584494295802448, penalty=l2, solver=liblinear; total time=   1.6s
[CV] END C=0.21584494295802448, penalty=l2, solver=liblinear; total time=   1.6s
[CV] END C=0.21584494295802448, penalty=l2, solver=liblinear; total time=   1.8s
[CV] END ...C=2.1333911067827613, penalty=no

In [39]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
import warnings
warnings.filterwarnings('ignore')

# Separate features and target variable
X = df.drop('Activity', axis=1)
y = df['Activity']

X = X.clip(lower=-1e6, upper=1e6)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Define the parameter distribution for hyperparameter tuning
param_dist = {
    'n_neighbors': randint(1, 31),  # Random integer range for n_neighbors
    'weights': ['uniform', 'distance'],  # Weight function used in prediction
    'p': [1, 2]  # Power parameter for the Minkowski metric
}

# Create the KNeighborsClassifier model
knn = KNeighborsClassifier()

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=knn, param_distributions=param_dist, n_iter=20, cv=10, scoring='roc_auc', random_state=42, n_jobs=-1)

# Fit the RandomizedSearchCV to find the best parameters
random_search.fit(X_train, y_train)

# Get the best parameters from RandomizedSearchCV
best_params = random_search.best_params_

# Train the KNeighborsClassifier model on the entire training set using the best parameters
best_knn = KNeighborsClassifier(**best_params)
best_knn.fit(X_train, y_train)

# Make predictions with the best model
y_pred = best_knn.predict(X_test)
y_pred_prob = best_knn.predict_proba(X_test)[:, 1]  # Probabilities for ROC AUC

# Compute metrics with the final model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
conf_matrix = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

# Extract values from confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Additional metrics
specificity = TN / (TN + FP)
balanced_accuracy = (recall + specificity) / 2
fpr = FP / (FP + TN)  # False Positive Rate

# Store the metrics in a list for visualization
model_metrics = {}
model_metrics['KNeighbors'] = [
    accuracy, roc_auc, precision, recall, f1, specificity, balanced_accuracy, fpr, mcc, [TN, FP, FN, TP]
]

# Beautified printing of metrics
print("\n")
print("="*40)
print("Model Evaluation Metrics".center(40))
print("="*40)
print(f"{'Metric':<20} {'Value':>18}")
print("-"*40)
print(f"{'Accuracy':<20} {accuracy:>18.4f}")
print(f"{'ROC AUC':<20} {roc_auc:>18.4f}")
print(f"{'Precision':<20} {precision:>18.4f}")
print(f"{'Recall':<20} {recall:>18.4f}")
print(f"{'F1 Score':<20} {f1:>18.4f}")
print(f"{'Specificity':<20} {specificity:>18.4f}")
print(f"{'Balanced Accuracy':<20} {balanced_accuracy:>18.4f}")
print(f"{'FPR':<20} {fpr:>18.4f}")
print(f"{'Matthews CC':<20} {mcc:>18.4f}")
print("\n")
print("="*40)
print("Confusion Matrix".center(40))
print("="*40)
print(f"{'TN':<10} {'FP':<10} {'FN':<10} {'TP':<10}")
print("-"*40)
print(f"{TN:<10} {FP:<10} {FN:<10} {TP:<10}")
print("="*40)
print("\n")


Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV] END ................var_smoothing=6.908929431882418e-10; total time=   0.2s
[CV] END ................var_smoothing=6.908929431882418e-10; total time=   0.2s
[CV] END ................var_smoothing=7.775643618422824e-10; total time=   0.3s
[CV] END ................var_smoothing=7.775643618422824e-10; total time=   0.2s
[CV] END ...............var_smoothing=1.1658782892785617e-10; total time=   0.2s
[CV] END ...............var_smoothing=1.1658782892785617e-10; total time=   0.3s
[CV] END .................var_smoothing=6.12093058299281e-10; total time=   0.3s
[CV] END .................var_smoothing=6.12093058299281e-10; total time=   0.2s
[CV] END .................var_smoothing=6.12093058299281e-10; total time=   0.3s
[CV] END ...............var_smoothing=3.2649577519793797e-10; total time=   0.2s
[CV] END ...............var_smoothing=3.2649577519793797e-10; total time=   0.2s
[CV] END ................var_smoothing=7.45172